In [1]:
import tensorflow as tf
import numpy as np
from tensorflow.keras import layers,Model
import pandas as pd
import re
from tensorflow.keras.callbacks import ModelCheckpoint,TensorBoard
from sklearn.preprocessing import StandardScaler
from sklearn.model_selection import train_test_split

In [2]:
data= pd.read_csv(r"C:\Users\bzhang\Desktop\processminer-rare-event-mts.csv")

In [3]:
data.head()

,time,y,x1,x2,x3,x4,x5,x6,x7,x8,...,x52,x53,x54,x55,x56,x57,x58,x59,x60,x61
0,5-1-99 0:00,0,0.376665,-4.596435,-4.095756,13.497687,-0.118830,-20.669883,0.000732,-0.061114,...,10.091721,0.053279,-4.936434,-24.590146,18.515436,3.473400,0.033444,0.953219,0.006076,0
1,5-1-99 0:02,0,0.475720,-4.542502,-4.018359,16.230659,-0.128733,-18.758079,0.000732,-0.061114,...,10.095871,0.062801,-4.937179,-32.413266,22.760065,2.682933,0.033536,1.090502,0.006083,0
2,5-1-99 0:04,0,0.363848,-4.681394,-4.353147,14.127998,-0.138636,-17.836632,0.010803,-0.061114,...,10.100265,0.072322,-4.937924,-34.183774,27.004663,3.537487,0.033629,1.840540,0.006090,0
3,5-1-99 0:06,0,0.301590,-4.758934,-4.023612,13.161567,-0.148142,-18.517601,0.002075,-0.061114,...,10.104660,0.081600,-4.938669,-35.954281,21.672449,3.986095,0.033721,2.554880,0.006097,0
4,5-1-99 0:08,0,0.265578,-4.749928,-4.333150,15.267340,-0.155314,-17.505913,0.000732,-0.061114,...,10.109054,0.091121,-4.939414,-37.724789,21.907251,3.601573,0.033777,1.410494,0.006105,0


In [4]:
first_derivatives = data.drop(columns = ['time','x61','x28','y'])

In [5]:
for i in range(1,len(first_derivatives)-1):
    first_derivatives.iloc[i,:] = (first_derivatives.iloc[i+1,:]-first_derivatives.iloc[i-1,:])/4

In [6]:
#second_derivatives = data.drop(columns = ['time','x61','x28','y'])
#for i in range(1,len(second_derivatives)-1):
    #second_derivatives.iloc[i,:] = (second_derivatives.iloc[i+1,:]-2*second_derivatives.iloc[i,:]+second_derivatives.iloc[i-1,:])/16

In [6]:
column_names=first_derivatives.columns.values.tolist()

In [7]:
first_difference_cols = dict(zip(column_names,[('d'+name) for name in column_names]))

In [8]:
first_derivatives=first_derivatives.rename(columns = first_difference_cols)

In [10]:
#second_difference_cols = dict(zip(column_names,[('dd'+name) for name in column_names]))

In [11]:
#second_derivatives=second_derivatives.rename(columns = second_difference_cols)

In [9]:
data.drop(columns = ['time'],inplace = True)
data_new_features = pd.concat([data,first_derivatives],axis=1)

In [10]:
data_new_features['dx28']=data_new_features['x28'].diff()

In [11]:
data_new_features.dropna(axis = 0,inplace = True)

In [12]:
for i in data_new_features.loc[data_new_features.y==1].index:
    data_new_features['y'][i]=0
    data_new_features['y'][i-2:i]=1

C:\ProgramData\Anaconda3\lib\site-packages\ipykernel_launcher.py:2: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame

See the caveats in the documentation: http://pandas.pydata.org/pandas-docs/stable/indexing.html#indexing-view-versus-copy
  
C:\ProgramData\Anaconda3\lib\site-packages\ipykernel_launcher.py:3: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame

See the caveats in the documentation: http://pandas.pydata.org/pandas-docs/stable/indexing.html#indexing-view-versus-copy
  This is separate from the ipykernel package so we can avoid doing imports until


In [13]:
data_new_features.shape

(18397, 122)

In [22]:
y = data_new_features['y']
x = data_new_features.drop(columns=['y'])
x_train,x_test,y_train,y_test = train_test_split(x,y,test_size = 0.1,shuffle = False)

In [23]:
training_data = x_train.loc[y_train==0]

In [24]:
test_data = x_test.loc[y_test==0]

In [25]:
scaler = StandardScaler().fit(training_data)

In [26]:
training_data=scaler.transform(training_data)
test_data=scaler.transform(test_data)

In [27]:
training_data=np.array(training_data,'float32')
test_data=np.array(test_data,'float32')

In [28]:
def prepare_tensor(data):
    sequence = []
    for i in range(len(data)-5):
        sequence.append(data[i:i+5])
    return sequence

In [29]:
training_data = prepare_tensor(training_data)
test_data = prepare_tensor(test_data)

In [30]:
training_data = tf.data.Dataset.from_tensor_slices(training_data).batch(64,drop_remainder=True)
test_data=tf.data.Dataset.from_tensor_slices(test_data).batch(64,drop_remainder=True)

In [31]:
class Encoder(layers.Layer):
    def __init__(self,units,seq_len):
        super(Encoder,self).__init__()
        self.lstm = layers.LSTM(units,activation = 'elu',recurrent_dropout=0.2,dropout = 0.2,return_sequences=True)
        self.bottle_neck = layers.LSTM(units//2, activation='elu',return_sequences = False,recurrent_dropout=0.2,dropout = 0.2)
        self.repeat = layers.RepeatVector(seq_len)
    
    def call(self,inputs):
        x = self.lstm(inputs)
        x = self.bottle_neck(x)
        x = self.repeat(x)
        return x

In [32]:
class Decoder(layers.Layer):
    def __init__(self,units,n_features):
        super(Decoder,self).__init__()
        self.lstm1 = layers.LSTM(units//2,activation = 'elu',recurrent_dropout=0.2,dropout = 0.2,return_sequences=True)
        self.lstm2 = layers.LSTM(units, activation='elu',return_sequences = True,recurrent_dropout=0.2,dropout = 0.2)
        self.decoder_output = layers.TimeDistributed(layers.Dense(n_features))
    
    def call(self,inputs):
        x = self.lstm1(inputs)
        x = self.lstm2(x)
        x = self.decoder_output(x)
        return x

In [33]:
class AutoEncoder(Model):
    def __init__(self,units,seq_len,n_features):
        super(AutoEncoder,self).__init__()
        self.encoder = Encoder(units,seq_len)
        self.decoder = Decoder(units,n_features)
    
    def call(self,inputs):
        x = self.encoder(inputs)
        y = self.decoder(x)
        return y

In [34]:
auto_encoder = AutoEncoder(64,5,121)

In [35]:
adam = tf.keras.optimizers.Adam(1e-4)

In [51]:
train_loss = tf.keras.metrics.Mean('train_loss', dtype=tf.float32)
test_loss = tf.keras.metrics.Mean('test_loss', dtype=tf.float32)

In [64]:
def train_step(model, optimizer, train_inputs):
    with tf.GradientTape() as tape:
        predictions = model(train_inputs)
        loss = tf.reduce_mean((train_inputs-predictions)**2)
    grads = tape.gradient(loss, model.trainable_variables)
    optimizer.apply_gradients(zip(grads, model.trainable_variables))

    train_loss(loss)
    
def test_step(model, test_inputs):
    predictions = model(test_inputs)
    loss = tf.reduce_mean((test_inputs-predictions)**2)

    test_loss(loss)

In [65]:
%reload_ext tensorboard

In [66]:
import datetime

In [67]:
current_time = datetime.datetime.now().strftime("%m%d-%H%M%S")
train_log_dir = 'logs/gradient_tape/' + current_time + '/train'
test_log_dir = 'logs/gradient_tape/' + current_time + '/test'
train_summary_writer = tf.summary.create_file_writer(train_log_dir)
test_summary_writer = tf.summary.create_file_writer(test_log_dir)

In [69]:
EPOCHS = 200
for epoch in range(EPOCHS):
    for (data) in training_data:
        train_step(auto_encoder,adam,data)
    with train_summary_writer.as_default():
        tf.summary.scalar('loss', train_loss.result(), step=epoch)

    for (data) in test_data:
        test_step(auto_encoder,data)
    with test_summary_writer.as_default():
        tf.summary.scalar('loss', test_loss.result(), step=epoch)
  
    template = 'Epoch {}, Loss: {}, Test Loss: {}'
    print (template.format(epoch+1,
                         train_loss.result(), 
                         test_loss.result()
                          ))
    train_loss.reset_states()
    test_loss.reset_states()

Epoch 1, Loss: 0.9462159872055054, Test Loss: 2.424018144607544
Epoch 2, Loss: 0.820109486579895, Test Loss: 2.2719712257385254
Epoch 3, Loss: 0.7617450952529907, Test Loss: 2.1903746128082275
Epoch 4, Loss: 0.7239757776260376, Test Loss: 2.1334216594696045
Epoch 5, Loss: 0.6866888403892517, Test Loss: 2.034834146499634
Epoch 6, Loss: 0.6529748439788818, Test Loss: 1.924087405204773
Epoch 7, Loss: 0.6238954663276672, Test Loss: 1.8008259534835815
Epoch 8, Loss: 0.5976119041442871, Test Loss: 1.7901045083999634
Epoch 9, Loss: 0.5719148516654968, Test Loss: 1.8715628385543823
Epoch 10, Loss: 0.5530669689178467, Test Loss: 1.5768619775772095
Epoch 11, Loss: 0.535690426826477, Test Loss: 1.4546785354614258
Epoch 12, Loss: 0.5183359980583191, Test Loss: 1.4626754522323608
Epoch 13, Loss: 0.5043005347251892, Test Loss: 1.314872145652771
Epoch 14, Loss: 0.48444414138793945, Test Loss: 1.234938621520996
Epoch 15, Loss: 0.47159141302108765, Test Loss: 1.1686197519302368
Epoch 16, Loss: 0.459993

Epoch 125, Loss: 0.19947224855422974, Test Loss: 0.5907110571861267
Epoch 126, Loss: 0.19318105280399323, Test Loss: 0.5986579060554504
Epoch 127, Loss: 0.18991929292678833, Test Loss: 0.5838176012039185
Epoch 128, Loss: 0.19118422269821167, Test Loss: 0.5859792828559875
Epoch 129, Loss: 0.18931294977664948, Test Loss: 0.596750795841217
Epoch 130, Loss: 0.18977122008800507, Test Loss: 0.5858073234558105
Epoch 131, Loss: 0.18919701874256134, Test Loss: 0.5547425150871277
Epoch 132, Loss: 0.18832166492938995, Test Loss: 0.5946952700614929
Epoch 133, Loss: 0.1893814206123352, Test Loss: 0.5840544104576111
Epoch 134, Loss: 0.18814371526241302, Test Loss: 0.5688008666038513
Epoch 135, Loss: 0.18588730692863464, Test Loss: 0.5661981701850891
Epoch 136, Loss: 0.18505701422691345, Test Loss: 0.5869277119636536
Epoch 137, Loss: 0.18408039212226868, Test Loss: 0.6085086464881897
Epoch 138, Loss: 0.18570666015148163, Test Loss: 0.5572432279586792
Epoch 139, Loss: 0.1855110228061676, Test Loss: 0.

In [126]:
healthy_cases = data_new_features.loc[data_new_features['y']==0]
analomies = data_new_features.loc[data_new_features['y']==1]
healthy_cases.drop(columns = ['y'],inplace = True)
analomies.drop(columns = ['y'],inplace = True)

C:\ProgramData\Anaconda3\lib\site-packages\pandas\core\frame.py:3940: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame

See the caveats in the documentation: http://pandas.pydata.org/pandas-docs/stable/indexing.html#indexing-view-versus-copy
  errors=errors)


In [127]:
healthy_cases=scaler.transform(healthy_cases)
analomies=scaler.transform(analomies)

In [128]:
healthy_cases=healthy_cases.astype('float32')

In [129]:
analomies = analomies.astype('float32')

In [130]:
healthy_cases = prepare_tensor(healthy_cases)
analomies = prepare_tensor(analomies)

In [131]:
analomies_loss = tf.keras.metrics.MeanSquaredError()
analomies = tf.data.Dataset.from_tensor_slices(analomies).batch(64,drop_remainder=True)
for x in analomies:
    prediction=auto_encoder(x)
    loss= analomies_loss(x,prediction)
print("analomies loss {}".format(loss))

analomies loss 3.0580782890319824


In [132]:
healthy_loss = tf.keras.metrics.MeanSquaredError()
healthy_cases = tf.data.Dataset.from_tensor_slices(healthy_cases).batch(64,drop_remainder=True)
for x in healthy_cases:
    prediction=auto_encoder(x)
    loss= healthy_loss(x,prediction)
print("healthy loss {}".format(loss))

healthy loss 0.2059439867734909
